In [71]:
import requests
import bs4
import pandas as pd
import time
import csv
import random
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from tkinter import *

In [72]:
df_Tenses = pd.read_csv(r'C:\Users\Jakub\Hiszpanski program\Lista_Czasow.csv',index_col=[0])
df_Verbs = pd.read_csv(r'C:\Users\Jakub\Hiszpanski program\Lista_slow.csv',index_col=[0])

In [73]:
#scraping all verbs from lingolex - part of df_Verbs creation

def scrapewordlist():
    listaurl = []
    #range is page amount
    for i in range(1):
        listaurl.append(str("https://lingolex.com/verbs/popular_verbs.php?page="+str(i)))

    wordlist = []
    lista =[]
    for i in range(len(listaurl)): 
        my_url = listaurl[i]
        uClient = uReq(my_url)
        page_html = uClient.read()
        #close connection
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        containers = page_soup.findAll("div", {"class":"col-sm-3 panel-heading"})
        
        i = 0
        while i < len(containers):
            wordlist.append(containers[i].text)
            i+=1


    i = 0
    del wordlist[0] #only spanish words
    sample = wordlist
    n = 4 # I want to iterate over every third element
    for i,x in enumerate(sample):
        if i % n == 0:
            lista.append(sample[i])
    #print(sample) #print everything from wordlist
    print(wordlist)
    return(lista)

In [74]:
def odmiany(verb):
    response = requests.get('https://dle.rae.es/'+verb)
    soup = BeautifulSoup(response.text,'html.parser')
    test = soup.find(class_="cnj")
    
    rows = []
    for td in test.find_all("td"):
        rows.append(td)    
    ####Tenses####
    
    Gerundio = rows[4].text
    
    Participio = rows[8].text
    
    Indicativo_Presente = [rows[12].text,rows[17].text,rows[22].text,rows[32].text,rows[37].text,rows[42].text]

    Indicativo_Preterito_Imperfecto = [rows[13].text,rows[18].text,rows[23].text,rows[33].text,rows[38].text,rows[43].text]

    Indicativo_Preterito_Perfecto_Simple = [rows[52].text,rows[57].text,rows[62].text,rows[72].text,rows[77].text,rows[82].text]

    Indicativo_Futuro_Simple = [rows[53].text,rows[58].text,rows[63].text,rows[73].text,rows[78].text,rows[83].text]
 
    Condicional_Simple = [rows[92].text,rows[96].text,rows[100].text,rows[108].text,rows[112].text,rows[116].text]

    Subjuntivo_Presente = [rows[124].text,rows[129].text,rows[134].text,rows[144].text,rows[149].text,rows[154].text]

    Subjuntivo_Futuro_Simple = [rows[125].text,rows[130].text,rows[135].text,rows[145].text,rows[150].text,rows[155].text]

    Preterito_Imperfecto = [rows[164].text,rows[168].text,rows[172].text,rows[180].text,rows[184].text,rows[188].text]

    Imperativo = [rows[196].text,rows[200].text,rows[204].text,rows[208].text]
    
    return(Gerundio,
            Participio,
            Indicativo_Presente,
            Indicativo_Preterito_Imperfecto,
            Indicativo_Preterito_Perfecto_Simple,
            Indicativo_Futuro_Simple,
            Condicional_Simple,
            Subjuntivo_Presente,
            Subjuntivo_Futuro_Simple,
            Preterito_Imperfecto,
            Imperativo)

In [ ]:
###DATA SCRAPING ODMIAN Z RAE.ES

start = time.time()
conjs_list =[]
error_list =[]
verb_list =[]
for word in range(len(df_Verbs)):
    try:
        #print(word,df_Verbs['word'][word])
        conjs_list.append(odmiany(df_Verbs['Slowo'][word]))
        verb_list.append(df_Verbs['Slowo'][word])
        print(word)
    except Exception:
        print(word,df_Verbs['Slowo'][word],'problem')
        link = 'https://dle.rae.es/'+ df_Verbs['Slowo'][word]
        resp = requests.get(link)
        soup1 = BeautifulSoup(resp.text,'html.parser')
        result = soup1.find("a",{"data-cat":"FETCH"})
        conjs_list.append(odmiany(result.text))
        error_list.append(df_Verbs['Slowo'][word])
        verb_list.append(result.text)
        pass
end = time.time()
print(end-start," seconds")

In [95]:
###DATASHEETS GENERATION FROM RAE.ES

Baza_Gerundio = []
Baza_Participio = []
Baza_Indicativo_Presente = []
Baza_Indicativo_Preterito_Imperfecto = []
Baza_Indicativo_Preterito_Perfecto_Simple = []
Baza_Indicativo_Futuro_Simple = []
Baza_Condicional_Simple = []
Baza_Subjuntivo_Presente = []
Baza_Subjuntivo_Futuro_Simple = []
Baza_Preterito_Imperfecto = []
Baza_Imperativo = []

for slowo in range(len(conjs_list)):
    Baza_Gerundio.append(conjs_list[slowo][0])
    Baza_Participio.append(conjs_list[slowo][1])
    Baza_Indicativo_Presente.append(conjs_list[slowo][3])
    Baza_Indicativo_Preterito_Imperfecto.append(conjs_list[slowo][4])
    Baza_Indicativo_Preterito_Perfecto_Simple.append(conjs_list[slowo][5])
    Baza_Indicativo_Futuro_Simple.append(conjs_list[slowo][2])
    Baza_Condicional_Simple.append(conjs_list[slowo][6])
    Baza_Subjuntivo_Presente.append(conjs_list[slowo][7])
    Baza_Subjuntivo_Futuro_Simple.append(conjs_list[slowo][8])
    Baza_Preterito_Imperfecto.append(conjs_list[slowo][9])
    Baza_Imperativo.append(conjs_list[slowo][10])
    
    cols = ["yo","tu","el/ella","nosotros","vosotros","ellos/ellas"]
    colsimp = ["tu","usted","vosotros","ustedes"]

pd.DataFrame(Baza_Gerundio, columns=["Forma"]).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Gerundio.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Participio, columns=["Forma"]).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Participio.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Indicativo_Presente, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Indicativo_Presente.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Indicativo_Preterito_Imperfecto, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Indicativo_Preterito_Imperfecto.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Indicativo_Preterito_Perfecto_Simple, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Indicativo_Preterito_Perfecto_Simple.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Indicativo_Futuro_Simple, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Indicativo_Futuro_Simple.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Condicional_Simple, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Condicional_Simple.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Subjuntivo_Presente, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Subjuntivo_Presente.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Subjuntivo_Futuro_Simple, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Subjuntivo_Futuro_Simple.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Preterito_Imperfecto, columns=cols).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Preterito_Imperfecto.csv',encoding='utf-8-sig')
pd.DataFrame(Baza_Imperativo, columns=colsimp).to_csv(path_or_buf = r'C:\Users\Jakub\Hiszpanski program\Odmiany\Baza_Imperativo.csv',encoding='utf-8-sig')